Evaluate Japanese tokenization using Amazon esci-data

Pre-requirement

* git clone esci-data
* Start Weaviate server

Index products to Weaviate

* only product_title

With GSE tokenization and Kagome JA tokenization.

1. create collection with GSE
2. index product_title and product_id



In [7]:
import os
import json
import weaviate
from weaviate.classes.config import Property, DataType, Tokenization, Configure

# initialize weaviate client
client = weaviate.connect_to_local(
    #host="weaviate",
    host="host.docker.internal",
)
if client.is_ready():
    print("Connected Weaviate server!")

# Create collection with tokenization properties
collection_name = "products"

if client.collections.exists(name=collection_name) == False :
    client.collections.create(
        name=collection_name,
        properties=[
            Property(
                name="product_id",
                data_type=DataType.TEXT,
                skip_vectorization=True,
                index_filterable=True,
                index_searchable=True
            ),
            Property(
                name="title_gse",
                data_type=DataType.TEXT,
                tokenization=Tokenization.GSE,
                skip_vectorization=True,
                index_filterable=True,
                index_searchable=True
            ),
            Property(
                name="title_kagome",
                data_type=DataType.TEXT,
                tokenization=Tokenization.KAGOME_JA,
                skip_vectorization=True,
                index_filterable=True,
                index_searchable=True
            )
        ],
        vectorizer_config=Configure.Vectorizer.none(),
        vector_index_config=Configure.VectorIndex.none()
    )

# Check collection schema
collection = client.collections.get(
    name=collection_name
)
print(collection)


Connected Weaviate server!
<weaviate.Collection config={
  "name": "Products",
  "description": null,
  "generative_config": null,
  "inverted_index_config": {
    "bm25": {
      "b": 0.75,
      "k1": 1.2
    },
    "cleanup_interval_seconds": 60,
    "index_null_state": false,
    "index_property_length": false,
    "index_timestamps": false,
    "stopwords": {
      "preset": "en",
      "additions": null,
      "removals": null
    }
  },
  "multi_tenancy_config": {
    "enabled": false,
    "auto_tenant_creation": false,
    "auto_tenant_activation": false
  },
  "properties": [
    {
      "name": "product_id",
      "description": null,
      "data_type": "text",
      "index_filterable": true,
      "index_range_filters": false,
      "index_searchable": true,
      "nested_properties": null,
      "tokenization": "word",
      "vectorizer_config": null,
      "vectorizer": "none"
    },
    {
      "name": "title_gse",
      "description": null,
      "data_type": "text",
   

In [8]:
import pandas as pd

df_products = pd.read_parquet('/workspace/esci-data/shopping_queries_dataset/shopping_queries_dataset_products.parquet')
df_products = df_products[df_products["product_locale"] == "jp"]

print(f"# of products = {len(df_products)}")

# of products = 339059


In [9]:
counter = 0
interval = 100000

collection = client.collections.get(
    name=collection_name
)

#with collection.batch.rate_limit(requests_per_minute=500) as batch:
with collection.batch.dynamic() as batch:
    for row in df_products.itertuples():
        if batch.number_errors > 10:
            pass
        product = row._asdict()
        properties = {
            "product_id": product["product_id"],
            "title_gse": product["product_title"],
            "title_kagome": product["product_title"]
        }
        batch.add_object(
            properties=properties
        )
        counter += 1
        if counter % interval == 0:
            print(f"Imported {counter} articles...")
    if batch.number_errors != 0 and len(client.batch.failed_objects) == batch.number_errors:
        print(f"Error during batch execution. {batch.number_errors} errors...")
    else :
        print(f"Finished importing {counter} articles.")



/workspace/.venv/lib/python3.12/site-packages/weaviate/warnings.py:303: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(
/usr/local/lib/python3.12/asyncio/selector_events.py:879: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=84 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)


Imported 100000 articles...
Imported 200000 articles...
Imported 300000 articles...
Finished importing 339059 articles.


In [10]:

collection.aggregate.over_all(total_count=True).total_count

339059

Prepare quries and judgements data

In [11]:
df_examples = pd.read_parquet('/workspace/esci-data/shopping_queries_dataset/shopping_queries_dataset_examples.parquet')
df_examples = df_examples[df_examples["product_locale"] == "jp"]
df_examples_products = pd.merge(
    df_examples,
    df_products,
    how='left',
    left_on=['product_locale','product_id'],
    right_on=['product_locale', 'product_id']
)
# query, query_id
queries: dict[str, str] = {}
# query_id, list(product_id, esci_label, split)
examples: dict[str, list[dict]] = {}

for row in df_examples_products.itertuples():
    product = row._asdict()
    tmp_query = product["query"]
    tmp_query_id = product["query_id"]
    queries[tmp_query_id] = tmp_query
    examples.setdefault(tmp_query_id, [])
    examples[tmp_query_id].append({
        "product_id": product["product_id"],
        "esci_label": product["esci_label"],
        "split": product["split"]
    })

print(f"Load {len(queries)} queries")


Load 18127 queries


Keyword search with Title

In [14]:
from weaviate.classes.query import MetadataQuery

k = 200

# filetr examples by esci_label == E
def exact_relevant_products(examples: list[dict]) -> list:
    relevants = []
    for item in examples:
        if item["esci_label"] == "E":
            relevants.append(item["product_id"])
    return relevants

# calculate recall @ k
def calc_recall(results: list, relevants: list) -> float:
    hits = 0
    for item in results:
        if item.properties["product_id"] in relevants:
            hits = hits+1
    return round(hits / len(relevants), 4)
    
# calculate precision @ k
def calc_precision(results: list, relevants: list) -> float:
    if not results:
        return 0.0
    hits = 0
    for item in results:
        if item.properties["product_id"] in relevants:
            hits = hits+1
    return round(hits / len(results), 4)

# count esci_label category
def count_esci_labels(labels:list) -> str:
    e = s = c = i = 0 
    for label in labels:
        if label["esci_label"] == "E":
            e = e+1
        elif label["esci_label"] == "S":
            s = s+1
        elif label["esci_label"] == "C":
            c = c+1
        elif label["esci_label"] == "I":
            i = i+1
    return f"E:{e}, S:{s}, C:{c}, I:{i}"


def debug_print(query, response, examples, relevants ,recall, precision):
    print(f"query [{query}]")
    print(f"# of objects : {len(response.objects)}")
    print(f"# of relevants : {len(examples[query_id])}")
    print(f"# of relevants : {len(relevants)}")
    print(f"recall    : {recall}")
    print(f"precision : {precision}")


# Recall / Precision for GSE

In [ ]:

field = "title_gse"

stats: list[dict] = []

for (query_id, query) in queries.items():
    response = collection.query.bm25(
        query=query,
        query_properties=[field],
        return_metadata=MetadataQuery(score=True),
        limit=k
    )
    relevants = exact_relevant_products(examples=examples[query_id])
    recall = calc_recall(results=response.objects, relevants=relevants)
    precision = calc_precision(results=response.objects, relevants=relevants)
    
    stats.append(
        {
            "query": query,
            "query_id": query_id,
            "k": k,
            "num_of_hits": len(response.objects),
            "num_of_relevants": len(relevants),
            "num_of_examples": len(examples[query_id]),
            "num_of_esci_labels": count_esci_labels(labels=examples[query_id]),
            "recall": recall,
            "precision": precision
        }
    )

pd_stats = pd.DataFrame(stats)
pd_stats



reacll describe.....
count    18127.000000
mean         0.647699
std          0.347901
min          0.000000
25%          0.363600
50%          0.750000
75%          1.000000
max          1.000000
Name: recall, dtype: float64
precision describe.....
count    18127.000000
mean         0.049369
std          0.062459
min          0.000000
25%          0.015000
50%          0.040000
75%          0.070000
max          1.000000
Name: precision, dtype: float64


,query,query_id,k,num_of_hits,num_of_relevants,num_of_examples,num_of_esci_labels,recall,precision
10869,ホウ酸,123384,200,10,16,16,"E:16, S:0, C:0, I:0",0.6250,1.0
13242,切腹,125757,200,1,1,16,"E:1, S:11, C:0, I:4",1.0000,1.0
14837,明恵,127353,200,15,16,16,"E:16, S:0, C:0, I:0",0.9375,1.0
15055,柳原白蓮,127571,200,14,15,16,"E:15, S:0, C:0, I:1",0.9333,1.0
15377,江副,127893,200,4,7,16,"E:7, S:3, C:0, I:6",0.5714,1.0
15537,清塚 信也,128053,200,1,16,16,"E:16, S:0, C:0, I:0",0.0625,1.0
15933,番線,128449,200,5,5,16,"E:5, S:6, C:0, I:5",1.0000,1.0
17533,雁が音,130054,200,2,10,16,"E:10, S:4, C:0, I:2",0.2000,1.0
17774,頭陀袋,130295,200,8,9,16,"E:9, S:7, C:0, I:0",0.8889,1.0
17906,鬼平,130428,200,16,16,16,"E:16, S:0, C:0, I:0",1.0000,1.0


In [13]:
import matplotlib.pyplot as plt

plt.hist(pd_stats["recall"], bins=100, label="recall", alpha=0.5)
plt.hist(pd_stats["precision"], bins=100, label="precision", alpha=0.5)
plt.legend()
plt.show()

print("reacll describe.....")
print(pd_stats["recall"].describe())
print("precision describe.....")
print(pd_stats["precision"].describe())
pd_stats[pd_stats["precision"] == 1.0]

# Recall / Precision for Kagome JA

In [15]:

field = "title_kagome"

stats: list[dict] = []

for (query_id, query) in queries.items():
    response = collection.query.bm25(
        query=query,
        query_properties=[field],
        return_metadata=MetadataQuery(score=True),
        limit=k
    )
    relevants = exact_relevant_products(examples=examples[query_id])
    recall = calc_recall(results=response.objects, relevants=relevants)
    precision = calc_precision(results=response.objects, relevants=relevants)
    
    stats.append(
        {
            "query": query,
            "query_id": query_id,
            "k": k,
            "num_of_hits": len(response.objects),
            "num_of_relevants": len(relevants),
            "num_of_examples": len(examples[query_id]),
            "num_of_esci_labels": count_esci_labels(labels=examples[query_id]),
            "recall": recall,
            "precision": precision
        }
    )

pd_stats = pd.DataFrame(stats)
pd_stats



,query,query_id,k,num_of_hits,num_of_relevants,num_of_examples,num_of_esci_labels,recall,precision
0,&ハニー シャンプー,116,200,200,19,40,"E:19, S:20, C:1, I:0",0.4737,0.045
1,'74ジェイルブレイク ac/dc,123,200,200,8,8,"E:8, S:0, C:0, I:0",0.8750,0.035
2,(マックバースト） mach burst 軽い コンフォート 穴開き スポーツ サドル 痛く...,190,200,200,1,40,"E:1, S:7, C:0, I:32",1.0000,0.005
3,(知っていることはあまりないけれど)家族です,191,200,200,1,40,"E:1, S:2, C:0, I:37",1.0000,0.005
4,- ビニール手袋粉なし,295,200,200,29,40,"E:29, S:11, C:0, I:0",0.8276,0.120
...,...,...,...,...,...,...,...,...,...
18122,ﾎﾙｽﾀｰ,130647,200,200,15,16,"E:15, S:0, C:1, I:0",0.0000,0.000
18123,ﾏｼﾞｯｸﾘﾝ,130648,200,200,16,16,"E:16, S:0, C:0, I:0",0.0000,0.000
18124,ﾒｽﾃｨﾝ,130649,200,200,11,16,"E:11, S:0, C:5, I:0",0.0000,0.000
18125,ﾚﾃﾞｨｰｽ水着,130650,200,200,16,16,"E:16, S:0, C:0, I:0",0.0000,0.000


In [ ]:
import matplotlib.pyplot as plt

plt.hist(pd_stats["recall"], bins=100, label="recall", alpha=0.5)
plt.hist(pd_stats["precision"], bins=100, label="precision", alpha=0.5)
plt.legend()
plt.show()

print("reacll describe.....")
print(pd_stats["recall"].describe())
print("precision describe.....")
print(pd_stats["precision"].describe())
pd_stats[pd_stats["precision"] == 1.0]

#### 以下、テスト・確認用のスクリプト

In [ ]:
df_examples = pd.read_parquet('/workspace/esci-data/shopping_queries_dataset/shopping_queries_dataset_examples.parquet')
df_examples[df_examples["query_id"] == 1]

In [ ]:
import pandas as pd

df_examples = pd.read_parquet('/workspace/esci-data/shopping_queries_dataset/shopping_queries_dataset_examples.parquet')
df_products = pd.read_parquet('/workspace/esci-data/shopping_queries_dataset/shopping_queries_dataset_products.parquet')
df_sources = pd.read_csv("/workspace/esci-data/shopping_queries_dataset/shopping_queries_dataset_sources.csv")
df_products = df_products[df_products["product_locale"] == "jp"]
df_examples = df_examples[df_examples["product_locale"] == "jp"]
df_examples_products = pd.merge(
    df_examples,
    df_products,
    how='left',
    left_on=['product_locale','product_id'],
    right_on=['product_locale', 'product_id']
)
df_examples_products_source = pd.merge(
    df_examples_products,
    df_sources,
    how='left',
    left_on=['query_id'],
    right_on=['query_id']
)
#df_examples_products_source["esci_label"].unique().tolist()
df_examples_products_source[df_examples_products_source["split"] == "test"]

# initialize weaviate client
client = weaviate.connect_to_local(
    host="weaviate",
    headers={
        "X-Google-Studio-Api-Key": os.getenv("GOOGLE_APIKEY")
    }
)
if client.is_ready():
    print("Connected Weaviate server!")

# Create collection with tokenization properties
collection_name = "products"

if client.collections.exists(name=collection_name) == False :
    client.collections.create(
        name=collection_name,
        vectorizer_config=[
            Configure.NamedVectors.text2vec_palm(
                name="title_vector", 
                source_properties=["title"],
                model_id="text-embedding-004",
                api_endpoint="generativelanguage.googleapis.com",
                project_id=""
            )
        ],
        properties=[
            Property(
                name="product_id",
                data_type=DataType.TEXT,
                index_filterable=True,
                index_searchable=True
            ),
            Property(
                name="title",
                data_type=DataType.TEXT,
                tokenization=Tokenization.GSE,
                index_filterable=True,
                index_searchable=True
            )
        ]
    )